In [51]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib

import random
import os

In [52]:

#---------------------------------------------------------------------------------------------#

# This function accepts the year as input and retrieves all the links of the files available in that year
def fetch_links(base_url, year):
    url = f"{base_url}{year}/"

    response = requests.get(url)

    base_url_year = base_url + year + '/'
    list_of_links = []
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a')
        
        for link in links:
            href = link.get('href')
            if href:
                absolute_url = urljoin(base_url_year, href)
                if ".csv" in absolute_url:
                    list_of_links.append(absolute_url)

        return list_of_links

In [53]:
# Function to randomly sample links from the list of all available links
random.seed(8)
def random_links(list_of_links, no_of_links):
    links_to_fetch = random.sample(list_of_links, no_of_links)
    return links_to_fetch

In [54]:
# Function to download all the required files using wget, and place it in the required folder
def download_files(links_to_fetch, folder):
    os.makedirs(folder, exist_ok=True)
        
    for url in links_to_fetch:
        filename = folder + "/" + url.split("/")[-1]
        urllib.request.urlretrieve(url, filename)

In [57]:
def create_folder(folder_path):
    if os.path.exists(folder_path):
        for root, dirs, files in os.walk(folder_path, topdown=False):
            for file in files:
                os.remove(os.path.join(root, file)) 
            for dir in dirs:
                os.rmdir(os.path.join(root, dir)) 
        os.rmdir(folder_path) 
    os.makedirs(folder_path) 

In [58]:
# Defining the base_url, year which we want to work with, number of files to be fetched and the destination folder for the csv files
base_url = "https://www.ncei.noaa.gov/data/local-climatological-data/access/"
year = "2023"
no_of_links = 5
folder = "../data/" + year
create_folder(folder)

In [59]:
list_of_links = fetch_links(base_url, year)
links_to_fetch = random_links(list_of_links, no_of_links)
download_files(links_to_fetch, folder)